## Part 1 : Using Term-document matrix for feature engineering and model was built using Random Forest Algorithm

In [1]:
import findspark

In [2]:
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
pyspark

<module 'pyspark' from '/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/__init__.py'>

In [5]:
from pyspark.sql import *

# Build the SparkSession
spark = SparkSession.builder \
   .appName("Assignment3p1") \
   .config("spark.some.config.option", "8gb") \
   .getOrCreate()
   


In [6]:
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [7]:
from pyspark.sql import SQLContext
sqlcontext=SQLContext(sc)

## Loading necessary libraries for this part

In [8]:
import numpy as np
import pandas as pd
import re
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
import re
# from pyspark.mllib.classification import LogisticRegressionWithSGD
# from pyspark.mllib.regression import LabeledPoint
# from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.sql import functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, VectorIndexer, IndexToString
from pyspark.mllib.linalg import SparseVector, DenseVector
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import RandomForestClassificationModel

## Loading training dataset

In [9]:
train_data = pd.read_csv("/home/cse587/Project3/train.csv",sep=",", header = 0)

Looking at the loaded training data

In [10]:
train_data.head()

,movie_id,movie_name,plot,genre
0,23890098,Taxi Blues,"Shlykov, a hard-working taxi driver and Lyosha...","['World cinema', 'Drama']"
1,31186339,The Hunger Games,The nation of Panem consists of a wealthy Capi...,"['Action/Adventure', 'Action', 'Science Fictio..."
2,20663735,Narasimham,Poovalli Induchoodan is sentenced for six yea...,"['Musical', 'Action', 'Drama']"
3,2231378,The Lemon Drop Kid,"The Lemon Drop Kid , a New York City swindler,...",['Comedy']
4,595909,A Cry in the Dark,Seventh-day Adventist Church pastor Michael Ch...,"['Crime Fiction', 'World cinema', 'Drama']"


# Converting pandas dataframe to pyspark dataframe

In [11]:
training_df = spark.createDataFrame(train_data)
training_df.head(1)

[Row(movie_id=23890098, movie_name='Taxi Blues', plot="Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.", genre="['World cinema', 'Drama']")]

Looking at the pyspark training dataframe 

In [14]:
training_df.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

## Cleaning the data

In [12]:
training_df= training_df.withColumn("plot",F.lower(F.col('plot')))

regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
data = regexTokenizer.transform(training_df)


In [13]:
extra_stopwords =['http','https','amp','rt','t','c','the','despite','working','develop','whence', 'here', 'show', 'were', 'why', 
            'n’t', 'the', 'whereupon', 'not', 'more', 'how', 'eight', 'indeed', 'i', 'only', 'via', 'nine', 
            're', 'themselves', 'almost', 'to', 'already', 'front', 'least', 'becomes', 'thereby', 'doing', 
            'her', 'together', 'be', 'often', 'then', 'quite', 'less', 'many', 'they', 'ourselves', 'take',
            'its', 'yours', 'each', 'would', 'may', 'namely', 'do', 'whose', 'whether', 'side', 'both', 'what',
            'between', 'toward', 'our', 'whereby', "'m", 'formerly', 'myself', 'had', 'really', 'call', 'keep',
            "'re", 'hereupon', 'can', 'their', 'eleven', '’m', 'even', 'around', 'twenty', 'mostly', 'did', 
            'at', 'an', 'seems', 'serious', 'against', "n't", 'except', 'has', 'five', 'he', 'last', '‘ve', 
            'because', 'we', 'himself', 'yet', 'something', 'somehow', '‘m', 'towards', 'his', 'six',
            'anywhere', 'us', '‘d', 'thru', 'thus', 'which', 'everything', 'become', 'herein', 'one', 'in',
            'although', 'sometime', 'give', 'cannot', 'besides', 'across', 'noone', 'ever', 'that', 'over', 
            'among', 'during', 'however', 'when', 'sometimes', 'still', 'seemed', 'get', "'ve", 'him', 'with', 
            'part', 'beyond', 'everyone', 'same', 'this', 'latterly', 'no', 'regarding', 'elsewhere', 'others', 
            'moreover', 'else', 'back', 'alone', 'somewhere', 'are', 'will', 'beforehand', 'ten', 'very',
            'most', 'three', 'former', '’re', 'otherwise', 'several', 'also', 'whatever', 'am', 'becoming', 
            'beside', '’s', 'nothing', 'some', 'since', 'thence', 'anyway', 'out', 'up', 'well', 'it',
            'various', 'four', 'top', '‘s', 'than', 'under', 'might', 'could', 'by', 'too', 'and',
            'whom', '‘ll', 'say', 'therefore', "'s", 'other', 'throughout', 'became', 'your', 'put', 'per', 
            "'ll", 'fifteen', 'must', 'before', 'whenever', 'anyone', 'without', 'does', 'was', 'where', 
            'thereafter', "'d", 'another', 'yourselves', 'n‘t', 'see', 'go', 'wherever', 'just', 'seeming', 
            'hence', 'full', 'whereafter', 'bottom', 'whole', 'own', 'empty', 'due', 'behind', 'while', 'onto', 
            'wherein', 'off', 'again', 'a', 'two', 'above', 'therein', 'sixty', 'those', 'whereas', 'using', 
            'latter', 'used', 'my', 'herself', 'hers', 'or', 'neither', 'forty', 'thereupon', 'now', 'after',
            'yourself', 'whither', 'rather', 'once', 'from', 'until', 'anything', 'few', 'into', 'such', 'being',
            'make', 'mine', 'please', 'along', 'hundred', 'should', 'below', 'third', 'unless', 'upon', 'perhaps',
            'ours', 'but', 'never', 'whoever', 'fifty', 'any', 'all', 'nobody', 'there', 'have', 'anyhow', 'of', 
            'seem', 'down', 'is', 'every', '’ll', 'much', 'none', 'further', 'me', 'who', 'nevertheless', 'about',
            'everywhere', 'name', 'enough', '’d', 'next', 'meanwhile', 'though', 'through', 'on', 'first', 'been', 
            'hereby', 'if', 'move', 'so', 'either', 'amongst', 'for', 'twelve', 'nor', 'she', 'always', 'these', 
            'as', '’ve', 'amount', '‘re', 'someone', 'afterwards', 'you', 'nowhere', 'itself', 'done', 'hereafter',
            'within', 'made', 'ca', 'them']

Removing the stop words

In [14]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered" ).setStopWords(extra_stopwords)
data = remover.transform(data)

## Generating Term-Document matrix for the modified data

In [15]:
countVectors = CountVectorizer(inputCol="filtered", outputCol="features",minDF=120)
data1 = countVectors.fit(data)
rescaledData = data1.transform(data)
#rescaledData.select('features').show()

In [19]:

rescaledData.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|               words|            filtered|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, t...|(4969,[8,107,449,...|
|31186339|    The Hunger Games|the nation of pan...|['Action/Adventur...|[the, nation, of,...|[nation, panem, c...|(4969,[0,4,5,8,12...|
|20663735|          Narasimham|poovalli induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, induch...|(4969,[0,1,6,8,16...|
| 2231378|  The Lemon Drop Kid|the lemon drop ki...|          ['Comedy']|[the, lemon, drop...|[lemon, drop, kid...|(4969,[0,5,7,9,12...|
|  595909|   A Cry in the Dark|seventh-da

Converting genre into labels using StringIndexer

In [16]:
label_stringIdx = StringIndexer(inputCol = "genre", outputCol = "label")
fitter = label_stringIdx.fit(rescaledData)
rescaledData = fitter.transform(rescaledData)
# rescaledData.show()

In [17]:
## Storing the labels
labels = fitter.labels

## Fitting Machine Learning model on the training dataset

In [20]:
## splitting training and testing data
(training_data,testing_data) = rescaledData.randomSplit([0.7,0.3], seed=123)

In [19]:
## Fitting Naive Bayes model on the training data

# from pyspark.ml.classification import NaiveBayes
# nb = NaiveBayes(smoothing=0.1)
# model = nb.fit(training_data)


In [21]:
## Fitting Random Forest model on the training data

from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=20, maxBins=100)
model = rf.fit(training_data)

## Saving the trained model

In [26]:
path1 = "/home/cse587/Project3/randomforest_for_part1"
model.save("/home/cse587/Project3/randomforest_for_part1")

## Loading the saved model

In [27]:
from pyspark.ml.classification import RandomForestClassificationModel
savedmodel = RandomForestClassificationModel.load(path1)

In [28]:
## fitting the obtained model on validation test data
predictions = savedmodel.transform(testing_data)


In [29]:
## Evaluating the accuracy of predicted labels

evaluator = MulticlassClassificationEvaluator(labelCol = "label", predictionCol="prediction")
evaluator.evaluate(predictions)

0.02669043003965693

In [30]:
predictions.select('movie_id','prediction').show()

+--------+----------+
|movie_id|prediction|
+--------+----------+
|    3217|       0.0|
|    3837|       0.0|
|    3947|       0.0|
|    4729|       0.0|
|    5313|       0.0|
|    9429|       0.0|
|    9835|       0.0|
|    9979|       0.0|
|   11992|       0.0|
|   11998|       0.0|
|   12001|       0.0|
|   12004|       0.0|
|   13154|       0.0|
|   17920|       0.0|
|   19701|       0.0|
|   20669|       0.0|
|   21174|       0.0|
|   22727|       0.0|
|   22751|       0.0|
|   23255|       0.0|
+--------+----------+
only showing top 20 rows



## Converting predicted labels to its corresponding predicted genre 

In [30]:
converter = IndexToString(inputCol="prediction", outputCol="originalCategory", labels = labels)
converted = converter.transform(predictions)


In [31]:
converted.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+----------+--------------------+
|movie_id|          movie_name|                plot|               genre|               words|            filtered|            features| label|       rawPrediction|         probability|prediction|    originalCategory|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+----------+--------------------+
|    3217|    Army of Darkness|after being pulle...|['Action/Adventur...|[after, being, pu...|[pulled, time, po...|(4969,[0,3,5,6,13...| 639.0|[-1377.3557989146...|[1.17345889846608...|      40.0|['Comedy', 'Horror']|
|    3837|     Blazing Saddles|in the american o...|          ['Comedy']|[in, the, america...|[american, old, w...|(4969,[0,2,7,

## Loading test data from local using Pandas

In [25]:

test_data = pd.read_csv("/home/cse587/Project3/test.csv",sep=",", header = 0)

## Creating testing dataframe in pyspark

In [26]:
testing_df = spark.createDataFrame(test_data)


## pre-processing the test data

In [27]:
testing_df= testing_df.withColumn("plot",F.lower(F.col('plot')))
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")

remover = StopWordsRemover(inputCol="words", outputCol="filtered" ).setStopWords(extra_stopwords)

countVectors = CountVectorizer(inputCol="filtered", outputCol="features",minDF=120)
pipeline = Pipeline(stages=[regexTokenizer, remover, countVectors])

data = pipeline.fit(training_df)
test_data = data.transform(testing_df)

In [45]:
test_data.show(1)

+--------+----------+--------------------+--------------------+--------------------+--------------------+
|movie_id|movie_name|                plot|               words|            filtered|            features|
+--------+----------+--------------------+--------------------+--------------------+--------------------+
| 1335380|    Exodus|the film is based...|[the, film, is, b...|[film, based, eve...|(4969,[0,1,2,5,6,...|
+--------+----------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



## Predicting the test labels by fitting the trained model

In [28]:
test_pred = model.transform(test_data)


In [29]:
test_pred.select('movie_id','prediction').show()

+--------+----------+
|movie_id|prediction|
+--------+----------+
| 1335380|       0.0|
|29062594|       0.0|
| 9252321|       0.0|
|13455076|       0.0|
|24165951|       0.0|
| 1925869|       0.0|
|10799612|       0.0|
|28238240|       0.0|
|17124781|       0.0|
|28207941|       0.0|
|19174305|       0.0|
|18392317|       0.0|
|34420857|       0.0|
| 4039635|       0.0|
| 8034072|       0.0|
| 4016437|       0.0|
| 1520023|       2.0|
|24589422|       0.0|
|35068740|       0.0|
|21132951|       0.0|
+--------+----------+
only showing top 20 rows



## Getting the original genre back from predicted labels

In [33]:
converter = IndexToString(inputCol="prediction", outputCol="originalCategory", labels = labels)
test_result = converter.transform(test_pred)
test_result.show(2)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------------+
|movie_id|          movie_name|                plot|               words|            filtered|            features|       rawPrediction|         probability|prediction|originalCategory|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------------+
| 1335380|              Exodus|the film is based...|[the, film, is, b...|[film, based, eve...|(4969,[0,1,2,5,6,...|[1.63401036216089...|[0.08170051810804...|       0.0|       ['Drama']|
|29062594|A la salida nos v...|a group of teenag...|[a, group, of, te...|[group, teenagers...|(4969,[5,6,44,57,...|[2.65486293857844...|[0.13274314692892...|       0.0|       ['Drama']|
+--------+--------------------+--------------------+------------------

## Loading the mapping.csv into a dataframe

In [4]:
mapping = pd.read_csv('/home/cse587/Project3/mapping.csv')
colnames = ['label','name']
mapping.columns =colnames
mapping_df = spark.createDataFrame(mapping)

mapping.head()

,label,name
0,0,Drama
1,1,Comedy
2,2,Romance Film
3,3,Thriller
4,4,Action


In [7]:
def convert(lines):
    lines = lines[1:-1].split(', ')
    temp=[0]*20
    for i in lines:   
        for k,v in mapping.iterrows():
            if i == ('\''+ v['name'] +'\''):
                temp[k] = 1
                continue
    return " ".join(map(str,temp))

converter = udf(convert)
test_result = test_result.withColumn("predictions",converter('originalCategory'))

In [66]:
test_result.show(1)

+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------------+--------------------+
|movie_id|movie_name|                plot|               words|            filtered|            features|       rawPrediction|         probability|prediction|originalCategory|    final_prediction|
+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------------+--------------------+
| 1335380|    Exodus|the film is based...|[the, film, is, b...|[film, based, eve...|(4969,[0,1,2,5,6,...|[-2135.1100553928...|[0.99907098773483...|       0.0|       ['Drama']|1 0 0 0 0 0 0 0 0...|
+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------------+--------------------+
only showing to

## Extracting the final dataframe

In [36]:
part1_result = test_result.select("movie_id","predictions")
# part1_result = part1_result.withColumnRenamed("final_prediction","predictions")
# part1_result.show(1)


## Saving the predicted results in .csv file

In [37]:
part1_result.write.csv('part1_test_result_rf.csv', header = True)

In [76]:
part1_result.head(1)

[Row(movie_id=1335380, predictions='1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0')]